In [16]:
import pandas as pd
import openai
from dotenv import load_dotenv
import os
import re

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")
pd.set_option('display.max_colwidth', None)

def extract_answer_tag(text: str) -> str:
    answer_match = re.search(r'<answer>(.*?)</answer>', text, re.DOTALL)
    answer_content = answer_match.group(1) if answer_match else None
    return answer_content

In [39]:
SQL_EVALUATION_PROMPT = """
You are a SQL expert. You are given a SQL query and a expected answer.
You need to evaluate if the SQL query is correct and if it answers the question.
You need to return a boolean value.
"""

SQL_CORRECTNESS_PROMPT = """
You are a SQL expert. You are given 2 results from a SQL query. One corresponds to the
gold query results, which are meant to be the truth. The other corresponds to another
query results.
You need to evaluate if the results from this other query is correct, meaning, if they
output the same results in terms of content. Even if they differ by commas, parenthesis
or other characters, you need to evaluate the actual content.
You need to return a boolean value.
"""


def call_gpt_api_to_check_query(sql_query, expected_answer, model="gpt-3.5-turbo"):
    prompt = """
    SQL query: {sql_query}
    Expected answer: {expected_answer}

    Return only the boolean value. In enclosed tags like <answer>true</answer> or <answer>false</answer>.
    """
    prompt = prompt.format(sql_query=sql_query, expected_answer=expected_answer)
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SQL_EVALUATION_PROMPT},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

def call_gpt_api_to_check_query_answer(sql_query_results, expected_query_answer, model="gpt-3.5-turbo"):
    prompt = """
    SQL query results: {sql_query}
    Gold query answer: {expected_answer}

    Return only the boolean value. In enclosed tags like <answer>true</answer> or <answer>false</answer>.
    """
    prompt = prompt.format(sql_query=sql_query_results, expected_answer=expected_query_answer)
    response = openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SQL_CORRECTNESS_PROMPT},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [26]:
import pandas as pd

df = pd.read_csv('exercise-3/test_file_results.csv')

df.head()

,question,expected_query,llm_query,llm_executed_query_results,expected_query_results
0,Liste os nomes dos restaurantes que ficam em Santo André.,select NomeR from Restaurante where CidadeR = 'Santo André';,"SELECT nomer AS ""Restaurant Name"" FROM restaurante WHERE cidader = 'Santo André';",Sabor de Minas,"[('Sabor de Minas',)]"
1,"Liste todas as informações dos produtos cujo nome começa com a letra “c” e cujo preço por quilo é inferior a R$3,00.",select * from Produto where (NomeP like 'c%' or NomeP like 'C%') and PrecoQuilo < 3;,"SELECT codp, nomep, precoquilo \nFROM produto \nWHERE lower(nomep) LIKE 'c%' \n AND precoquilo < 3.00;",2003: cebola (max length: 2.98)\n2004: cenoura (max length: 1.98)\n2005: chuchu (max length: 2.49),"[(2003, 'cebola', Decimal('2.98')), (2004, 'cenoura', Decimal('1.98')), (2005, 'chuchu', Decimal('2.49'))]"
2,Liste os códigos dos agricultores que já entregaram produtos para o restaurante RU-USP.,select CodA from Entrega natural join Restaurante where NomeR = 'RU-USP';,SELECT DISTINCT a.coda \nFROM agricultor a \nJOIN entrega e ON a.coda = e.coda \nJOIN restaurante r ON e.codr = r.codr \nWHERE r.nomer = 'RU-USP' \nORDER BY a.coda;,"1001, 1004","[(1004,), (1001,)]"
3,Liste os nomes dos produtos que já foram alguma vez entregues por um agricultor de Mogi das Cruzes.,select distinct NomeP from Entrega natural join Produto natural join Agricultor where CidadeA = 'Mogi das Cruzes';,"SELECT DISTINCT p.nomep, p.precoquilo \nFROM entrega e \nJOIN agricultor a ON e.coda = a.coda \nJOIN produto p ON e.codp = p.codp \nWHERE a.cidadea = 'Mogi das Cruzes' \nORDER BY p.nomep;","[('alface', Decimal('2.98')), ('batata', Decimal('0.98')), ('cebola', Decimal('2.98')), ('chuchu', Decimal('2.49')), ('couve-flor', Decimal('3.90'))]","[('alface',), ('batata',), ('cebola',), ('chuchu',), ('couve-flor',)]"
4,Liste os códigos dos agricultores que já entregaram batatas e também já entregaram cebolas.,select CodA from Entrega natural join Produto where NomeP = 'batata' intersect select CodA from Entrega natural join Produto where NomeP = 'cebola';,SELECT DISTINCT a.coda \nFROM agricultor a \nWHERE EXISTS ( \n SELECT 1 \n FROM entrega e \n INNER JOIN produto p ON e.codp = p.codp \n WHERE e.coda = a.coda AND p.nomep = 'batatas' \n) \nAND EXISTS ( \n SELECT 1 \n FROM entrega e \n INNER JOIN produto p ON e.codp = p.codp \n WHERE e.coda = a.coda AND p.nomep = 'cebolas' \n);,NaN,"[(1002,), (1004,), (1003,)]"


In [27]:
df['llm_query'] = df['llm_query'].str.replace('\n', ' ')
df['llm_query'] = df['llm_query'].str.lower()
df['expected_query'] = df['expected_query'].str.replace('\n', ' ')
df['expected_query'] = df['expected_query'].str.lower()

In [28]:
df.head()

,question,expected_query,llm_query,llm_executed_query_results,expected_query_results
0,Liste os nomes dos restaurantes que ficam em Santo André.,select nomer from restaurante where cidader = 'santo andré';,"select nomer as ""restaurant name"" from restaurante where cidader = 'santo andré';",Sabor de Minas,"[('Sabor de Minas',)]"
1,"Liste todas as informações dos produtos cujo nome começa com a letra “c” e cujo preço por quilo é inferior a R$3,00.",select * from produto where (nomep like 'c%' or nomep like 'c%') and precoquilo < 3;,"select codp, nomep, precoquilo from produto where lower(nomep) like 'c%' and precoquilo < 3.00;",2003: cebola (max length: 2.98)\n2004: cenoura (max length: 1.98)\n2005: chuchu (max length: 2.49),"[(2003, 'cebola', Decimal('2.98')), (2004, 'cenoura', Decimal('1.98')), (2005, 'chuchu', Decimal('2.49'))]"
2,Liste os códigos dos agricultores que já entregaram produtos para o restaurante RU-USP.,select coda from entrega natural join restaurante where nomer = 'ru-usp';,select distinct a.coda from agricultor a join entrega e on a.coda = e.coda join restaurante r on e.codr = r.codr where r.nomer = 'ru-usp' order by a.coda;,"1001, 1004","[(1004,), (1001,)]"
3,Liste os nomes dos produtos que já foram alguma vez entregues por um agricultor de Mogi das Cruzes.,select distinct nomep from entrega natural join produto natural join agricultor where cidadea = 'mogi das cruzes';,"select distinct p.nomep, p.precoquilo from entrega e join agricultor a on e.coda = a.coda join produto p on e.codp = p.codp where a.cidadea = 'mogi das cruzes' order by p.nomep;","[('alface', Decimal('2.98')), ('batata', Decimal('0.98')), ('cebola', Decimal('2.98')), ('chuchu', Decimal('2.49')), ('couve-flor', Decimal('3.90'))]","[('alface',), ('batata',), ('cebola',), ('chuchu',), ('couve-flor',)]"
4,Liste os códigos dos agricultores que já entregaram batatas e também já entregaram cebolas.,select coda from entrega natural join produto where nomep = 'batata' intersect select coda from entrega natural join produto where nomep = 'cebola';,select distinct a.coda from agricultor a where exists ( select 1 from entrega e inner join produto p on e.codp = p.codp where e.coda = a.coda and p.nomep = 'batatas' ) and exists ( select 1 from entrega e inner join produto p on e.codp = p.codp where e.coda = a.coda and p.nomep = 'cebolas' );,NaN,"[(1002,), (1004,), (1003,)]"


In [32]:
auditor_answers = []
for i, item in df.iterrows():
    sql_query = item['llm_query']
    expected_answer = item['expected_query']
    result = call_gpt_api_to_check_query(sql_query, expected_answer)
    auditor_answers.append(result)

df['auditor_answer'] = auditor_answers
df.auditor_answer = df.auditor_answer.apply(lambda x: extract_answer_tag(x))
df.to_csv('exercise-3/test_file_results_auditor.csv', index=False)

In [33]:
df.head()

,question,expected_query,llm_query,llm_executed_query_results,expected_query_results,auditor_answer
0,Liste os nomes dos restaurantes que ficam em Santo André.,select nomer from restaurante where cidader = 'santo andré';,"select nomer as ""restaurant name"" from restaurante where cidader = 'santo andré';",Sabor de Minas,"[('Sabor de Minas',)]",false
1,"Liste todas as informações dos produtos cujo nome começa com a letra “c” e cujo preço por quilo é inferior a R$3,00.",select * from produto where (nomep like 'c%' or nomep like 'c%') and precoquilo < 3;,"select codp, nomep, precoquilo from produto where lower(nomep) like 'c%' and precoquilo < 3.00;",2003: cebola (max length: 2.98)\n2004: cenoura (max length: 1.98)\n2005: chuchu (max length: 2.49),"[(2003, 'cebola', Decimal('2.98')), (2004, 'cenoura', Decimal('1.98')), (2005, 'chuchu', Decimal('2.49'))]",false
2,Liste os códigos dos agricultores que já entregaram produtos para o restaurante RU-USP.,select coda from entrega natural join restaurante where nomer = 'ru-usp';,select distinct a.coda from agricultor a join entrega e on a.coda = e.coda join restaurante r on e.codr = r.codr where r.nomer = 'ru-usp' order by a.coda;,"1001, 1004","[(1004,), (1001,)]",false
3,Liste os nomes dos produtos que já foram alguma vez entregues por um agricultor de Mogi das Cruzes.,select distinct nomep from entrega natural join produto natural join agricultor where cidadea = 'mogi das cruzes';,"select distinct p.nomep, p.precoquilo from entrega e join agricultor a on e.coda = a.coda join produto p on e.codp = p.codp where a.cidadea = 'mogi das cruzes' order by p.nomep;","[('alface', Decimal('2.98')), ('batata', Decimal('0.98')), ('cebola', Decimal('2.98')), ('chuchu', Decimal('2.49')), ('couve-flor', Decimal('3.90'))]","[('alface',), ('batata',), ('cebola',), ('chuchu',), ('couve-flor',)]",false
4,Liste os códigos dos agricultores que já entregaram batatas e também já entregaram cebolas.,select coda from entrega natural join produto where nomep = 'batata' intersect select coda from entrega natural join produto where nomep = 'cebola';,select distinct a.coda from agricultor a where exists ( select 1 from entrega e inner join produto p on e.codp = p.codp where e.coda = a.coda and p.nomep = 'batatas' ) and exists ( select 1 from entrega e inner join produto p on e.codp = p.codp where e.coda = a.coda and p.nomep = 'cebolas' );,NaN,"[(1002,), (1004,), (1003,)]",true


In [46]:
df.auditor_answer.value_counts()

auditor_answer
true     10
false     6
Name: count, dtype: int64

In [35]:
df.auditor_answer.value_counts(normalize=True) * 100

auditor_answer
true     62.5
false    37.5
Name: proportion, dtype: float64

In [40]:
auditor_query_answers = []
for i, item in df.iterrows():
    sql_query = item['llm_executed_query_results']
    expected_answer = item['expected_query_results']
    result = call_gpt_api_to_check_query_answer(sql_query, expected_answer)
    auditor_query_answers.append(result)

df['auditor_query_results_answer'] = auditor_query_answers
df.auditor_query_results_answer = df.auditor_query_results_answer.apply(lambda x: extract_answer_tag(x))
df.to_csv('exercise-3/test_file_results_auditor.csv', index=False)

In [43]:
df.head()

,question,expected_query,llm_query,llm_executed_query_results,expected_query_results,auditor_answer,auditor_query_results_answer
0,Liste os nomes dos restaurantes que ficam em Santo André.,select nomer from restaurante where cidader = 'santo andré';,"select nomer as ""restaurant name"" from restaurante where cidader = 'santo andré';",Sabor de Minas,"[('Sabor de Minas',)]",false,true
1,"Liste todas as informações dos produtos cujo nome começa com a letra “c” e cujo preço por quilo é inferior a R$3,00.",select * from produto where (nomep like 'c%' or nomep like 'c%') and precoquilo < 3;,"select codp, nomep, precoquilo from produto where lower(nomep) like 'c%' and precoquilo < 3.00;",2003: cebola (max length: 2.98)\n2004: cenoura (max length: 1.98)\n2005: chuchu (max length: 2.49),"[(2003, 'cebola', Decimal('2.98')), (2004, 'cenoura', Decimal('1.98')), (2005, 'chuchu', Decimal('2.49'))]",false,true
2,Liste os códigos dos agricultores que já entregaram produtos para o restaurante RU-USP.,select coda from entrega natural join restaurante where nomer = 'ru-usp';,select distinct a.coda from agricultor a join entrega e on a.coda = e.coda join restaurante r on e.codr = r.codr where r.nomer = 'ru-usp' order by a.coda;,"1001, 1004","[(1004,), (1001,)]",false,true
3,Liste os nomes dos produtos que já foram alguma vez entregues por um agricultor de Mogi das Cruzes.,select distinct nomep from entrega natural join produto natural join agricultor where cidadea = 'mogi das cruzes';,"select distinct p.nomep, p.precoquilo from entrega e join agricultor a on e.coda = a.coda join produto p on e.codp = p.codp where a.cidadea = 'mogi das cruzes' order by p.nomep;","[('alface', Decimal('2.98')), ('batata', Decimal('0.98')), ('cebola', Decimal('2.98')), ('chuchu', Decimal('2.49')), ('couve-flor', Decimal('3.90'))]","[('alface',), ('batata',), ('cebola',), ('chuchu',), ('couve-flor',)]",false,false
4,Liste os códigos dos agricultores que já entregaram batatas e também já entregaram cebolas.,select coda from entrega natural join produto where nomep = 'batata' intersect select coda from entrega natural join produto where nomep = 'cebola';,select distinct a.coda from agricultor a where exists ( select 1 from entrega e inner join produto p on e.codp = p.codp where e.coda = a.coda and p.nomep = 'batatas' ) and exists ( select 1 from entrega e inner join produto p on e.codp = p.codp where e.coda = a.coda and p.nomep = 'cebolas' );,NaN,"[(1002,), (1004,), (1003,)]",true,false


In [47]:
df.auditor_query_results_answer.value_counts()

auditor_query_results_answer
true     11
false     5
Name: count, dtype: int64

In [44]:
df.auditor_query_results_answer.value_counts(normalize=True) * 100

auditor_query_results_answer
true     68.75
false    31.25
Name: proportion, dtype: float64